In [1]:
%matplotlib inline
import copy
from qutip import *
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

import utilities as use
from stateobj import QState, QAncilla, JointSystem

from IPython.display import Markdown, display

In [2]:
UNITARY_EVOLUTION = False  # Slower
MEQ_EVOLUTION = True
SIM_ID = None  # string (system dimentions)
PLOT = True

TIMESTEPS = 10000
TIMEDELTA = 1e-02
OMEGA = 10  # Strength of Interaction

## System

In [3]:
system_dimentions = 10
system_energy = 1
system1 = use.create_system('fock', n_dims=system_dimentions, n=0)
system2 = use.create_system('fock', n_dims=system_dimentions, n=1)
# Tensor Product state
system = JointSystem([system1, system2])
type(system)

stateobj.JointSystem

## Ancilla

In [4]:
alpha = complex(1/math.sqrt(2), 0)
beta = complex(1/math.sqrt(2), 0)
phi = np.pi/3
eta = QAncilla(alpha, beta, phi)
# Stampa informazioni sull'Ancella
display(eta)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.5  +0.j         0.   +0.j         0.   +0.j        ]
 [0.   +0.j         0.25 +0.j         0.125+0.21650635j]
 [0.   +0.j         0.125-0.21650635j 0.25 +0.j        ]]

## Interaction

In [5]:
ap_int = [tensor(system.ap[i], qeye(3)) for i in range(len(system.systems))]
am_int = [tensor(system.am[i], qeye(3)) for i in range(len(system.systems))]
# Ladder Operators for the Ancilla
bp_int = tensor(qeye(system_dimentions), qeye(system_dimentions), eta.sigmaminus)
bm_int = tensor(qeye(system_dimentions), qeye(system_dimentions), eta.sigmaplus)
# Interaction 
V = [OMEGA*(ap_int[i]*bp_int + am_int[i]*bm_int) for i in range(len(system.systems))]
# Unitary Evolution Operator
U = [(-1j*V[i]*TIMEDELTA).expm() for i in range(len(system.systems))]

In [6]:
for i in tqdm(range(TIMESTEPS)):
    joint_system = tensor(system1, system2)
    total_system = use.cascade_evolution(eta, joint_system, [V[0], V[1]])
    system1, system2 = total_system.ptrace([0]), total_system.ptrace([1])
    
tensor(system1, system2)

100%|█████████████████████████████████████| 10000/10000 [22:00<00:00,  7.57it/s]


Quantum object: dims = [[10, 10], [10, 10]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

In [7]:
# Single Systems
display(system1)
display(system2)

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

In [8]:
for i in tqdm(range(TIMESTEPS)):
    system = system.interact(eta, V, TIMEDELTA)
    
system

100%|█████████████████████████████████████| 10000/10000 [23:58<00:00,  6.95it/s]


Quantum object: dims = [[10, 10], [10, 10]], shape = (100, 100), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

In [9]:
display(system.systems[0])
display(system.systems[1])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

# Master Equation Evolution
$$
    \frac{\Delta \rho_{n}}{\Delta t} = -i[H_S'+H_S''; \rho_{n-1}] + \mathcal{D}_S[\rho_{n-1}]
$$

$$
\begin{align}
    &H_S' = \langle \rangle \\
    &H_S'' = \sum_\pm\langle [B_+;B_-] \rangle A_{1+}A_{2-} \\
    &\mathcal{D}_S[\rho] = 
\end{align}
$$